In [19]:
!ls

NLP-JFK-nightrose.ipynb  README.md   test.csv	      titles.csv
readme.md		 readme.txt  titles_1000.csv  train.csv


In [62]:
import pandas as pd

#!mkdir /Data
#!aws s3 cp s3://nlp-workshop-reviews/amazon_review_polarity_csv.tgz /Data
!tar -xvzf s3://nlp-amazon-review/amazon_review_polarity_csv.tgz
#df = pd.read_csv("s3://nlp-amazon-review/amazon_review_polarity_csv/train.csv", names=["Label", "Title", "Review"])

tar (child): Cannot connect to s3: resolve failed

gzip: stdin: unexpected end of file
tar: Child returned status 128
tar: Error is not recoverable: exiting now


In [20]:
import pandas as pd
df = pd.read_csv("train.csv", names=["Label", "Title", "Review"])
df.shape

(3600000, 3)

In [15]:
df.head()

,Label,Title,Review
0,2,Stuning even for the non-gamer,This sound track was beautiful! It paints the ...
1,2,The best soundtrack ever to anything.,I'm reading a lot of reviews saying that this ...
2,2,Amazing!,This soundtrack is my favorite music of all ti...
3,2,Excellent Soundtrack,I truly like this soundtrack and I enjoy video...
4,2,"Remember, Pull Your Jaw Off The Floor After He...","If you've played the game, you know how divine..."


In [16]:
df.Label.unique()

array([2, 1])

In [17]:
df[df.Label==1].head()

,Label,Title,Review
6,1,Buyer beware,"This is a self-published book, and if you want..."
10,1,The Worst!,A complete waste of time. Typographical errors...
13,1,Oh please,I guess you have to be a romance novel lover f...
14,1,Awful beyond belief!,I feel I have to write to keep others from was...
15,1,Don't try to fool us with fake reviews.,It's glaringly obvious that all of the glowing...


In [18]:
df.Label.value_counts()

2    1800000
1    1800000
Name: Label, dtype: int64

In [4]:
import boto3
import json

comprehend = boto3.client(service_name='comprehend', region_name='us-east-1')
                
text = "It is raining today in Seattle"

print('Calling DetectSentiment')
result=json.dumps(comprehend.detect_sentiment(Text=text, LanguageCode='en'), sort_keys=True, indent=4)
print('End of DetectSentiment\n')



Calling DetectSentiment
End of DetectSentiment



In [33]:
text = "It is raining today in Seattle"

print('Calling DetectSentiment')
result=comprehend.detect_sentiment(Text=text,LanguageCode='en')
#json.dumps(comprehend.detect_sentiment(Text=text, LanguageCode='en'), sort_keys=True, indent=4)
print('End of DetectSentiment\n')

Calling DetectSentiment
End of DetectSentiment



In [36]:
result['Sentiment']

'NEUTRAL'

In [37]:
titles=df.Title.tolist()

In [71]:
reviews=df.Review.tolist()

In [103]:
pd.DataFrame(titles[0:1000]).to_csv('titles_1000.csv')

In [104]:
!aws s3 cp titles_1000.csv s3://nlp-amazon-review/title_1000.csv

upload: ./titles_1000.csv to s3://nlp-amazon-review/title_1000.csv


In [46]:
import numpy as np
titles_clean=['NAN' if x is np.nan else x for x in titles]

In [48]:
titles_clean[2372607]

'NAN'

In [54]:
def call_batch(inputs):
    response = comprehend.batch_detect_sentiment(
        TextList=inputs,
        LanguageCode='en'
    )

    return [x['Sentiment'] for x in response['ResultList']]

In [ ]:
total_output_from_review

In [72]:
total_output=[]
total_output_from_review=[]
for i in range(1000):#round(len(titles_clean)/25)):
    if i%100==0:
        print(i)
    batch_idx=0+i*25
    total_output.extend(call_batch(titles_clean[batch_idx:batch_idx+25]))
    total_output_from_review.extend(call_batch(reviews[batch_idx:batch_idx+25]))

0
100
200
300
400
500


KeyboardInterrupt: 

In [75]:
title_output=[1 if x =='NEGATIVE' else 2 for x in total_output]
review_output=[1 if x =='NEGATIVE' else 2 for x in total_output_from_review]

In [25]:
responses = comprehend.start_sentiment_detection_job(
    InputDataConfig={
        'S3Uri': 's3://nlp-amazon-review/title_1000.csv',
        'InputFormat': 'ONE_DOC_PER_LINE'
    },
    OutputDataConfig={
        'S3Uri': 's3://nlp-amazon-review/predict_from_title_1000.csv'#,
       # 'KmsKeyId': 'string'
    },
    DataAccessRoleArn='arn:aws:iam::360629003875:role/service-role/AmazonSageMaker-ExecutionRole-20190716T180741',
    JobName='sentiment classification',
    LanguageCode='en'
)

In [26]:
responses#=comprehend.describe_sentiment_detection_job(JobId= 'f325bc23d1925c577ccc726a996e1ae8')

{'JobId': '4ea82b9ea6a03c18c121b14caddc7c23',
 'JobStatus': 'SUBMITTED',
 'ResponseMetadata': {'RequestId': 'f7b7e5f3-bdc7-49f6-bb57-fc8f4625b26f',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'f7b7e5f3-bdc7-49f6-bb57-fc8f4625b26f',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '68',
   'date': 'Thu, 19 Sep 2019 14:41:07 GMT'},
  'RetryAttempts': 0}}

In [15]:
responses['JobStatus']#['SentimentDetectionJobProperties']['JobStatus']

'SUBMITTED'

In [ ]:
import time

import datetime
 

while True: 
    time.sleep(5)
    currentDT = datetime.datetime.now()
    #responses=comprehend.describe_sentiment_detection_job(JobId= 'f325bc23d1925c577ccc726a996e1ae8')
    status=responses['JobStatus']#['SentimentDetectionJobProperties']['JobStatus']
    if status in ['SUBMITTED','IN_PROGRESS']:#'COMPLETE':
        print(currentDT,status)
    elif status=='COMPLETED':
        print("complete!")
    else:
        break

2019-09-19 14:41:37.243308 SUBMITTED
2019-09-19 14:41:42.244815 SUBMITTED
2019-09-19 14:41:47.249963 SUBMITTED
2019-09-19 14:41:52.255306 SUBMITTED
2019-09-19 14:41:57.260462 SUBMITTED
2019-09-19 14:42:02.265812 SUBMITTED
2019-09-19 14:42:07.271332 SUBMITTED
2019-09-19 14:42:12.276483 SUBMITTED
2019-09-19 14:42:17.281821 SUBMITTED
2019-09-19 14:42:22.287237 SUBMITTED
2019-09-19 14:42:27.292577 SUBMITTED
2019-09-19 14:42:32.297742 SUBMITTED
2019-09-19 14:42:37.303093 SUBMITTED
2019-09-19 14:42:42.308437 SUBMITTED
2019-09-19 14:42:47.313589 SUBMITTED
2019-09-19 14:42:52.316200 SUBMITTED
2019-09-19 14:42:57.321534 SUBMITTED
2019-09-19 14:43:02.326680 SUBMITTED
2019-09-19 14:43:07.332019 SUBMITTED
2019-09-19 14:43:12.337536 SUBMITTED
2019-09-19 14:43:17.342688 SUBMITTED
2019-09-19 14:43:22.347840 SUBMITTED
2019-09-19 14:43:27.350531 SUBMITTED
2019-09-19 14:43:32.354149 SUBMITTED
2019-09-19 14:43:37.354674 SUBMITTED
2019-09-19 14:43:42.356629 SUBMITTED
2019-09-19 14:43:47.358574 SUBMITTED
2

In [8]:
!aws s3 cp s3://nlp-amazon-review/predict_from_title_1000.csv ./

fatal error: An error occurred (404) when calling the HeadObject operation: Key "predict_from_title_1000.csv" does not exist


In [12]:
!aws s3 ls s3://nlp-amazon-review/

                           PRE predict_from_title.csv/
                           PRE predict_from_title_1000.csv/
2019-09-18 19:13:58  643695014 amazon_review_full_csv.tgz
2019-09-18 19:24:57       1538 readme.txt
2019-09-18 19:24:57  176046679 test.csv
2019-09-18 20:50:36      29394 title_1000.csv
2019-09-18 19:41:10  120288528 titles.csv
2019-09-18 19:24:57 1585200224 train.csv
